In [1]:
import pandas as pd
import numpy as np

In [2]:
mappings = {
    '1000M':'1Gbps (GigE)',
    '1G':'1Gbps (GigE)',
    'GIGE':'1Gbps (GigE)',
    'GIG':'1Gbps (GigE)',
    'GIG-E':'1Gbps (GigE)',
    'GBPS':'1Gbps (GigE)',
    '10M': '10Mbps',
    '10 M': '10Mbps',
    'FET': '100 Mbps (FastE)',
    '100M': '100 Mbps (FastE)',
    '10 GigE': '10 Gbps (10 GigE)',
}

In [3]:
file = 'dummy.csv'
df = pd.read_csv(file)
df['bandwidth'] = df.apply(lambda row: str(row['bandwidth']), axis=1)
columns = list(df.columns.values)
df.head()

,Site ID,NPA-MXX,Interface,Facility Type,CLLI Code,bandwidth,Access Level,bandwidth.1,Access Level.1,bandwidth.2,Access Level.2
0,Site ID 1,NPA-MXX 1,Interface 1,Facility Type 1,CLLI Code,1 Gig/100MB,$100,1.5Gbps,$200,2 Gig,$300
1,Site ID 2,NPA-MXX 2,Interface 2,Facility Type 2,CLLI Code,10Mbps,$101,20Mbps,$201,50Mbps,$301
2,Site ID 3,NPA-MXX 3,Interface 3,Facility Type 3,CLLI Code,10Mbps,$102,20mMbps,$202,NaN,NaN
3,Site ID 4,NPA-MXX 4,Interface 4,Facility Type 4,CLLI Code,1000M,$103,100Mbps,$203,NaN,NaN
4,Site ID 5,NPA-MXX 5,Interface 5,Facility Type 5,CLLI Code,10Mbps,$104,20Mbps,$204,10M,$304


In [4]:
def combine_parallel_columns(df):
    df_o = df[['Site ID', 'NPA-MXX', 'Interface', 'Facility Type', 'CLLI Code', 'bandwidth', 'Access Level']].copy()
    df_1 = df[['Site ID', 'NPA-MXX', 'Interface', 'Facility Type', 'CLLI Code', 'bandwidth.1', 'Access Level.1']].copy()
    df_1.rename(columns = {
        'bandwidth.1':'bandwidth',
        'Access Level.1':'Access Level'
    }, inplace=True)

    df_2 = df[['Site ID', 'NPA-MXX', 'Interface', 'Facility Type', 'CLLI Code', 'bandwidth.2', 'Access Level.2']].copy()
    df_2.rename(columns = {
        'bandwidth.2':'bandwidth',
        'Access Level.2':'Access Level'
    }, inplace=True)
    df = pd.concat([df_o, df_1])
    df = pd.concat([df, df_2])
    return df

In [5]:
def get_repeated_col_and_mappings(columns):
    rep = []
    mappings = {}
    
    for col in columns:
        for c in columns:
            if c != col and c.startswith(col):
                rep.append(col)
                mappings[c] = col
    
    return list(set(rep)), mappings

def get_distinct_repeated_and_mappings_of_col(columns):
    repeat, mapping = get_repeated_col_and_mappings(columns)
    distinct= [col for col in columns if col not in mapping.keys()]    
    return repeat, mapping, distinct
    
def combine_parallel_tables(df):
    columns = list(df.columns.values)
    repeat, mappings, distinct = get_distinct_repeated_and_mappings_of_col(columns)
    
    df_disinct = df[distinct].copy()
    num_new_df_created = len(mappings) / len(repeat)
    
    for i in range(1, int(num_new_df_created)+1):
        req_col = list(set(distinct) - set(repeat)) + [col for col in columns if col.endswith(str(i))]
        df_temp = df[req_col].copy()
        df_temp.rename(columns = mappings, inplace=True)
        df_disinct = pd.concat([df_disinct, df_temp], sort=True)
        
    return df_disinct.reset_index()

def explode_df(df, char=r'/'):
    list_of_series = [(row["Site ID"], row["NPA-MXX"], row["Interface"], row["Facility Type"], row["CLLI Code"], row["Access Level"], str(row['bandwidth']).split(char)) for _, row in df.iterrows()]
    df_1 = pd.DataFrame(list_of_series, columns=['Site ID', 'NPA-MXX', 'Interface', 'Facility Type', 'CLLI Code', 'Access Level', 'bandwidth'])
    s = df_1.apply(lambda x: pd.Series(x['bandwidth']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'bandwidth_new'
    df_1 = df_1.drop('bandwidth', axis=1).join(s).reset_index()
    df_1.rename(columns = {'bandwidth_new': 'bandwidth'}, inplace=True)
    return df_1

In [6]:
df = combine_parallel_tables(df)
df.head()

,index,Access Level,CLLI Code,Facility Type,Interface,NPA-MXX,Site ID,bandwidth
0,0,$100,CLLI Code,Facility Type 1,Interface 1,NPA-MXX 1,Site ID 1,1 Gig/100MB
1,1,$101,CLLI Code,Facility Type 2,Interface 2,NPA-MXX 2,Site ID 2,10Mbps
2,2,$102,CLLI Code,Facility Type 3,Interface 3,NPA-MXX 3,Site ID 3,10Mbps
3,3,$103,CLLI Code,Facility Type 4,Interface 4,NPA-MXX 4,Site ID 4,1000M
4,4,$104,CLLI Code,Facility Type 5,Interface 5,NPA-MXX 5,Site ID 5,10Mbps


In [7]:
df = explode_df(df, char=r'/')
df.head()

,index,Site ID,NPA-MXX,Interface,Facility Type,CLLI Code,Access Level,bandwidth
0,0,Site ID 1,NPA-MXX 1,Interface 1,Facility Type 1,CLLI Code,$100,1 Gig
1,0,Site ID 1,NPA-MXX 1,Interface 1,Facility Type 1,CLLI Code,$100,100MB
2,1,Site ID 2,NPA-MXX 2,Interface 2,Facility Type 2,CLLI Code,$101,10Mbps
3,2,Site ID 3,NPA-MXX 3,Interface 3,Facility Type 3,CLLI Code,$102,10Mbps
4,3,Site ID 4,NPA-MXX 4,Interface 4,Facility Type 4,CLLI Code,$103,1000M


In [8]:
df.bandwidth.replace(to_replace=mappings, inplace=True)
df.head()

,index,Site ID,NPA-MXX,Interface,Facility Type,CLLI Code,Access Level,bandwidth
0,0,Site ID 1,NPA-MXX 1,Interface 1,Facility Type 1,CLLI Code,$100,1 Gig
1,0,Site ID 1,NPA-MXX 1,Interface 1,Facility Type 1,CLLI Code,$100,100MB
2,1,Site ID 2,NPA-MXX 2,Interface 2,Facility Type 2,CLLI Code,$101,10Mbps
3,2,Site ID 3,NPA-MXX 3,Interface 3,Facility Type 3,CLLI Code,$102,10Mbps
4,3,Site ID 4,NPA-MXX 4,Interface 4,Facility Type 4,CLLI Code,$103,1Gbps (GigE)
